In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Définir les chemins des dossiers
base_dir = r"C:\Users\Ons\Downloads\dataset_2\dataset"  # Dossier principal de ton dataset
output_dir = r"C:\Users\Ons\Downloads\dataset_output"  # Dossier où seront stockées les images divisées

# Fonction pour lister les images et leurs labels (classes)
def list_images_from_dir(base_dir):
    image_paths = []
    image_labels = []
    for label in os.listdir(base_dir):  # Liste des sous-dossiers (classes)
        label_dir = os.path.join(base_dir, label)
        if os.path.isdir(label_dir):  # Vérifier que c'est bien un dossier
            for img_file in os.listdir(label_dir):  # Liste des fichiers dans chaque sous-dossier
                if img_file.endswith(('.jpg', '.png')):  # Vérifier que ce sont bien des images
                    image_paths.append(os.path.join(label_dir, img_file))
                    image_labels.append(label)
    return image_paths, image_labels

# Liste les images et leurs labels dans le dossier `train`
train_dir = os.path.join(base_dir, 'train')  # Ton dossier train
test_dir = os.path.join(base_dir, 'test')  # Ton dossier test
train_images, train_labels = list_images_from_dir(train_dir)

# Diviser les images en 80% train, 10% validation, 10% test
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, stratify=train_labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, stratify=y_val, random_state=42)

# Créer la structure des dossiers pour le split (train, val, test)
os.makedirs(os.path.join(output_dir, 'train', 'others'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'train', 'ordonnance'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'train', 'bulletin_de_soin'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val', 'others'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val', 'ordonnance'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val', 'bulletin_de_soin'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test', 'others'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test', 'ordonnance'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test', 'bulletin_de_soin'), exist_ok=True)

# Fonction pour copier les images dans le bon dossier
def copy_images(image_paths, image_labels, split_type):
    for img_path, label in zip(image_paths, image_labels):
        # Définir le chemin de destination
        dest_dir = os.path.join(output_dir, split_type, label)
        shutil.copy(img_path, dest_dir)  # Copier l'image dans le dossier correspondant

# Copier les images dans les sous-dossiers train, val, test
copy_images(X_train, y_train, 'train')
copy_images(X_val, y_val, 'val')
copy_images(X_test, y_test, 'test')

print("Séparation terminée avec succès.")


Séparation terminée avec succès.


In [2]:
!pip install tensorflow


In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models

# Paramètres
img_size = 299  # Xception nécessite des images de taille 299x299
batch_size = 32
epochs = 10

# Création de générateurs d'images pour la data augmentation et la normalisation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalisation des pixels
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Préparer les générateurs d'images pour les données d'entraînement et de validation
train_generator = train_datagen.flow_from_directory(
    os.path.join(output_dir, 'train'),  # Dossier train où les images sont stockées
    target_size=(img_size, img_size),  # Redimensionner les images à la taille Xception
    batch_size=batch_size,
    class_mode='categorical'  # Les étiquettes sont dans plusieurs classes
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(output_dir, 'val'),  # Dossier validation
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# Charger le modèle Xception pré-entraîné (sans la couche de classification finale)
base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Geler les poids du modèle de base pour ne pas les réentraîner
base_model.trainable = False

# Ajouter une nouvelle tête de réseau (couches de classification)
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(3, activation='softmax')  # 3 classes : others, ordonnance, bulletin_de_soin
])

# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

# Sauvegarder le modèle entraîné
model.save("xception_model.h5")


Found 1315 images belonging to 3 classes.
Found 164 images belonging to 3 classes.
 5111808/83683744 ━━━━━━━━━━━━━━━━━━━━ 9:49 8us/step

KeyboardInterrupt: 

In [15]:
import os
import shutil
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models

# Définir les chemins des dossiers
base_dir = r"C:\Users\Ons\Downloads\dataset_2\dataset"  # Dossier principal de ton dataset
output_dir = r"C:\Users\Ons\Downloads\dataset_output"  # Dossier où seront stockées les images divisées

# Fonction pour lister les images et leurs labels (classes)
def list_images_from_dir(base_dir):
    image_paths = []
    image_labels = []
    for label in os.listdir(base_dir):  # Liste des sous-dossiers (classes)
        label_dir = os.path.join(base_dir, label)
        if os.path.isdir(label_dir):  # Vérifier que c'est bien un dossier
            for img_file in os.listdir(label_dir):  # Liste des fichiers dans chaque sous-dossier
                if img_file.endswith(('.jpg', '.png')):  # Vérifier que ce sont bien des images
                    image_paths.append(os.path.join(label_dir, img_file))
                    image_labels.append(label)
    return image_paths, image_labels

# Liste les images et leurs labels dans le dossier `train`
train_dir = os.path.join(base_dir, 'train')  # Ton dossier train
test_dir = os.path.join(base_dir, 'test')  # Ton dossier test
train_images, train_labels = list_images_from_dir(train_dir)

# Diviser les images en 80% train, 10% validation, 10% test
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, stratify=train_labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, stratify=y_val, random_state=42)

# Prétraitement des images avec ImageDataGenerator
img_size = 299  # Taille d'entrée de Xception (299x299)
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    os.path.join(output_dir, 'val'),
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(output_dir, 'test'),
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical'
)

# Charger le modèle Xception pré-entraîné
base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Geler les couches du modèle pour éviter de les réentraîner
base_model.trainable = False

# Ajouter une couche de classification
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(3, activation='softmax')  # Nombre de classes (3 dans ton cas)
])

# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    verbose=1
)

# Évaluer sur les données de test
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")


Found 2794 images belonging to 3 classes.
Found 164 images belonging to 3 classes.
Found 165 images belonging to 3 classes.
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 56s 1us/step


C:\Users\Ons\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 635s 7s/step - accuracy: 0.7199 - loss: 0.6876 - val_accuracy: 0.9268 - val_loss: 0.2085
Epoch 2/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 688s 8s/step - accuracy: 0.9254 - loss: 0.2616 - val_accuracy: 0.9695 - val_loss: 0.1193
Epoch 3/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 587s 7s/step - accuracy: 0.9347 - loss: 0.2125 - val_accuracy: 0.9756 - val_loss: 0.0941
Epoch 4/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 598s 7s/step - accuracy: 0.9403 - loss: 0.1904 - val_accuracy: 0.9573 - val_loss: 0.1100
Epoch 5/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 536s 6s/step - accuracy: 0.9517 - loss: 0.1569 - val_accuracy: 0.9512 - val_loss: 0.1106
Epoch 6/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 1453s 17s/step - accuracy: 0.9509 - loss: 0.1579 - val_accuracy: 0.9573 - val_loss: 0.0937
Epoch 7/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 524s 6s/step - accuracy: 0.9488 - loss: 0.1563 - val_accuracy: 0.9634 - val_loss: 0.0829
Epoch 8/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 526s 6s/step - accuracy: 0.9495 - loss: 0.1351 - val_accuracy: 0.9817 -

In [21]:

# Sauvegarder le modèle après l'entraînement
model.save('mon_modele.keras')  # Sauvegarde du modèle


In [37]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# 1. Charger ton modèle pré-entraîné
model = tf.keras.models.load_model('mon_modele.keras')  # Remplace par le chemin vers ton modèle sauvegardé

# 2. Charger et pré-traiter l'image à tester
img_path = r'C:\Users\Ons\Documents\696278-fond-noir-texture-gratuit-vectoriel.jpg'  # Remplace par le chemin vers l'image à tester

img = image.load_img(img_path, target_size=(299, 299))  # Taille conforme au modèle Xception

# Convertir l'image en tableau numpy
img_array = image.img_to_array(img)

# Ajouter une dimension pour le batch (1, 299, 299, 3)
img_array = np.expand_dims(img_array, axis=0)

# Normaliser l'image
img_array = img_array / 255.0

# Faire la prédiction
prediction = model.predict(img_array)
print(f"Prédiction : {prediction}")




# Afficher le nom de la classe prédite
predicted_class = np.argmax(prediction, axis=1)  # Indice de la classe prédite
predicted_class_name = class_names[predicted_class[0]]  # Nom de la classe
print(f"Nom de la classe prédite : {predicted_class_name}")
print(f"Classe prédite : {predicted_class}")




1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prédiction : [[0.00592459 0.11845969 0.8756157 ]]
Nom de la classe prédite : Other
Classe prédite : [2]


In [33]:
# Afficher les noms des classes
print(class_names)

['Bulletin_de_soin', 'Ordonnance', 'Other']


In [39]:
###ENTRE ESSAI 

In [5]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models

# Définir les chemins des dossiers
base_dir = r"C:\Users\Ons\Downloads\dataset_2\dataset"
output_dir = r"C:\Users\Ons\Downloads\dataset_output"

# Prétraitement des images avec ImageDataGenerator
img_size = 299  # Taille d'entrée de Xception (299x299)
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    os.path.join(base_dir, 'val'),
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical'
)

# Charger le modèle Xception pré-entraîné
base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Geler les couches du modèle pour éviter de les réentraîner
base_model.trainable = False

# Ajouter des couches personnalisées pour la classification
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')  # 3 classes
])

# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# **Première phase d'entraînement** : avec les couches gelées
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    verbose=1
)

# **Fine-Tuning** : On dégèle les dernières couches du modèle Xception
base_model.trainable = True

# **Débloquer seulement les dernières couches**
for layer in base_model.layers[:-10]:  # Geler toutes les couches sauf les 10 dernières
    layer.trainable = False

# **Recompiler avec un plus petit taux d'apprentissage**
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),  # LR plus petit
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# **Deuxième phase d'entraînement** : avec fine-tuning sur les couches dégivrées
history_finetuned = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    verbose=1
)

# **Évaluer sur les données de test**
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

# **Sauvegarder le modèle fine-tuné**
model.save('mon_modele_finetuned.keras')


Found 2794 images belonging to 3 classes.


FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'C:\\Users\\Ons\\Downloads\\dataset_2\\dataset\\val'

In [1]:
#Autre essai


In [4]:
import os
import shutil
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models

# 📌 1. Définir les chemins
base_dir = r"C:\Users\Ons\Downloads\dataset_2\dataset"
output_dir = r"C:\Users\Ons\Downloads\dataset_output"

# 📌 2. Fonction pour lister les images et leurs classes
def list_images_from_dir(directory):
    image_paths = []
    image_labels = []
    if not os.path.exists(directory):
        print(f"Le dossier {directory} n'existe pas !")
        return [], []

    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for img_file in os.listdir(label_dir):
                if img_file.endswith(('.jpg', '.png', '.jpeg')):
                    image_paths.append(os.path.join(label_dir, img_file))
                    image_labels.append(label)
    return image_paths, image_labels

# 📌 3. Vérifier l'existence des dossiers et recréer si nécessaire
os.makedirs(output_dir, exist_ok=True)
for split in ['train', 'val', 'test']:
    for category in ['others', 'ordonnance', 'bulletin_de_soin']:
        os.makedirs(os.path.join(output_dir, split, category), exist_ok=True)

# 📌 4. Charger les images et labels
train_images, train_labels = list_images_from_dir(os.path.join(base_dir, 'train'))

# 📌 5. Diviser les images (80% train, 10% val, 10% test)
X_train, X_temp, y_train, y_temp = train_test_split(train_images, train_labels, test_size=0.2, stratify=train_labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# 📌 6. Copier les images dans les nouveaux dossiers
def copy_images(image_paths, image_labels, split_type):
    for img_path, label in zip(image_paths, image_labels):
        dest_dir = os.path.join(output_dir, split_type, label)
        shutil.copy(img_path, dest_dir)

copy_images(X_train, y_train, 'train')
copy_images(X_val, y_val, 'val')
copy_images(X_test, y_test, 'test')

print("✔ Séparation des données terminée !")

# 📌 7. Définir les générateurs de données
img_size = 299
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(output_dir, 'train'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    os.path.join(output_dir, 'val'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(output_dir, 'test'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# 📌 8. Charger le modèle Xception pré-entraîné
base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False  # On gèle les couches du modèle

# 📌 9. Construire le modèle final
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')  # 3 classes : others, ordonnance, bulletin_de_soin
])

# 📌 10. Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 📌 11. Première phase d'entraînement (avec couches gelées)
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    verbose=1
)

# 📌 12. Fine-tuning : dégeler les 10 dernières couches de Xception
base_model.trainable = True
for layer in base_model.layers[:-10]:  
    layer.trainable = False

# 📌 13. Recompiler avec un plus petit taux d'apprentissage
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 📌 14. Deuxième phase d'entraînement (fine-tuning)
history_finetuned = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    verbose=1
)

# 📌 15. Évaluation sur les données de test
test_loss, test_acc = model.evaluate(test_generator)
print(f"✅ Test Accuracy: {test_acc:.4f}")

# 📌 16. Sauvegarder le modèle
model.save('mon_modele_finetuned.keras')
print("✔ Modèle sauvegardé avec succès !")


✔ Séparation des données terminée !
Found 1315 images belonging to 3 classes.
Found 1644 images belonging to 3 classes.
Found 573 images belonging to 3 classes.


C:\Users\Ons\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 607s 14s/step - accuracy: 0.8353 - loss: 0.2931 - val_accuracy: 0.9939 - val_loss: 0.0258
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 678s 16s/step - accuracy: 0.9944 - loss: 0.0262 - val_accuracy: 0.9854 - val_loss: 0.0416
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 767s 18s/step - accuracy: 0.9954 - loss: 0.0139 - val_accuracy: 0.9824 - val_loss: 0.0499
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 696s 17s/step - accuracy: 0.9942 - loss: 0.0137 - val_accuracy: 0.9976 - val_loss: 0.0152
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 705s 17s/step - accuracy: 0.9906 - loss: 0.0219 - val_accuracy: 0.9945 - val_loss: 0.0130
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 700s 17s/step - accuracy: 0.9948 - loss: 0.0129 - val_accuracy: 0.9945 - val_loss: 0.0127
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 698s 17s/step - accuracy: 0.9992 - loss: 0.0051 - val_accuracy: 0.9964 - val_loss: 0.0092
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 697s 17s/step - accuracy: 0.9974 - loss: 0.0097 - val_accuracy: 0.

In [6]:
from tensorflow.keras.models import load_model

# Charger le modèle sauvegardé
model = load_model('mon_modele_finetuned.keras')
print("✔ Modèle chargé avec succès !")


✔ Modèle chargé avec succès !


In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Définir la taille de l'image attendue par le modèle
img_size = 299  

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(img_size, img_size))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Ajouter une dimension batch
    img_array /= 255.  # Normalisation
    return img_array

# Exemple d'image à tester
image_path = r"C:\Users\Ons\Downloads\doxaria\doxaria\042ef63e-ccb0-4397-8b32-081b725badcc.png"
img_array = preprocess_image(image_path)

# Faire une prédiction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)  # Récupérer la classe avec la probabilité max

# Définir les classes en fonction du training
class_names = ['others', 'ordonnance', 'bulletin_de_soin']

# Afficher la prédiction
print(f"✅ Prédiction : {class_names[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
✅ Prédiction : others


In [ ]:
#autre essai 3

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# 📌 1. Définir les chemins
output_dir = r"C:\Users\Ons\Downloads\dataset_output"

# 📌 2. Paramètres
img_size = 300
batch_size = 32
num_classes = 3  # others, ordonnance, bulletin_de_soin

# 📌 3. Définir les générateurs de données
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(output_dir, 'train'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    os.path.join(output_dir, 'val'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(output_dir, 'test'),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# 📌 4. Charger EfficientNetB3 (FULL TRAINABLE dès le départ)
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = True  # 🔥 On entraîne tout dès le départ

# 📌 5. Construire le modèle
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),  # 📌 Stabilisation
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.4),  # 📌 Réduction du surajustement
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')  # 3 classes
])

# 📌 6. Compiler le modèle
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# 📌 7. Callbacks pour optimiser l'entraînement
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", mode="min")

# 📌 8. Entraînement du modèle (UNE SEULE PHASE)
history = model.fit(
    train_generator,
    epochs=10,  # 🔹 Max 20 epochs, mais stoppé automatiquement si nécessaire
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

# 📌 9. Charger le meilleur modèle et évaluer
model.load_weights("best_model.keras")
test_loss, test_acc = model.evaluate(test_generator)
print(f"✅ Test Accuracy: {test_acc:.4f}")

# 📌 10. Sauvegarde du modèle final
model.save('efficientnetb3_final.keras')
print("✔ Modèle sauvegardé avec succès !")
